In [1]:
import numpy as np
import pandas as pd

coef_nclaims_df = pd.read_csv("./out/coef_nclaims.txt", delimiter=";")
coef_nclaims_df.rename(lambda col_name: col_name + "_N", axis=1, inplace=True)
coef_nclaims_df

,Estimate_N,Std. Error_N,t value_N,Pr(>|t|)_N
(Intercept),-1.679381,0.126209,-13.306391,2.489291e-40
zoneD,0.233977,0.068960,3.392962,6.919475e-04
zoneE,0.406756,0.070018,5.809270,6.312527e-09
zoneF,0.415336,0.156986,2.645689,8.155014e-03
zoneA;B,-0.165548,0.069769,-2.372787,1.765822e-02
brand2,0.055325,0.066262,0.834944,4.037534e-01
brand6,-0.117159,0.133800,-0.875628,3.812366e-01
brand12,-0.133241,0.083152,-1.602375,1.090791e-01
brand14,-0.213623,0.328277,-0.650739,5.152178e-01
brand10;11;13,0.264279,0.097379,2.713933,6.651231e-03


In [2]:
coef_sev_df = pd.read_csv("./out/coef_sev.txt", delimiter=";")
coef_sev_df.rename(lambda col_name: col_name + "_Y", axis=1, inplace=True)
coef_sev_df

,Estimate_Y,Std. Error_Y,t value_Y,Pr(>|t|)_Y
(Intercept),7.277705,0.091878,79.210579,0.000000
power5,-0.075189,0.062591,-1.201280,0.229801
power6,-0.141010,0.059376,-2.374888,0.017659
power7,-0.102386,0.060176,-1.701434,0.089034
power8,-0.050447,0.084812,-0.594810,0.552046
power9,-0.276241,0.097512,-2.832880,0.004665
power10,-0.083611,0.092518,-0.903732,0.366259
power11,-0.110996,0.129698,-0.855803,0.392221
power12,-0.231768,0.173883,-1.332900,0.182733
power13,0.094797,0.220609,0.429706,0.667461


Join both dataframes by index...

In [3]:
def renamer_function(name):
    new_name = name
    if "(Intercept)" in name:
        new_name = "Standard Insured"
    elif "zone" in name:
        new_name = f"Zone {name[4:]}"
    elif "brand" in name:
        new_name = f"Brand {name[5:]}"
    elif "fuel" in name:
        new_name = f"Fuel {name[4:]}"
    elif "agecut" in name:
        new_name = f"Age {name[7:-1]}".replace(",","-")
    elif "vehcut" in name:
        new_name = f"Vehicle Age {name[7:-1]}".replace(",","-")
    elif "power" in name:
        new_name = f"Power {name[5:]}"
    
    #remove ; from name and return new name
    return new_name.replace(';', ' & ')

pricing_structure_df = pd.concat([coef_nclaims_df, coef_sev_df], axis=1)[['Estimate_N', 'Estimate_Y']] \
    .fillna(0) \
    .rename(columns={"Estimate_N": "beta_N", "Estimate_Y": "beta_Y"}) \
    .rename(renamer_function)
pricing_structure_df


,beta_N,beta_Y
Standard Insured,-1.679381,7.277705
Zone D,0.233977,0.000000
Zone E,0.406756,0.000000
Zone F,0.415336,0.000000
Zone A & B,-0.165548,0.000000
Brand 2,0.055325,0.043495
Brand 6,-0.117159,-0.071770
Brand 12,-0.133241,0.127862
Brand 14,-0.213623,-0.146288
Brand 10 & 11 & 13,0.264279,0.024495


Now let's compute E(N) and E(Y)

In [4]:
std_beta_N, std_beta_Y = pricing_structure_df.loc['Standard Insured'][0], pricing_structure_df.loc['Standard Insured'][1]
print("Standard Insured -> ", "beta N:", std_beta_N, "; beta Y:", std_beta_Y)

def E_N(row):
    if row.name != "Standard Insured":
        return np.exp(row.beta_N + std_beta_N)
    else:
        return np.exp(std_beta_N)
    
def E_Y(row):
    if row.name != "Standard Insured":
        return np.exp(row.beta_Y + std_beta_Y)
    else:
        return np.exp(std_beta_Y)

    
pricing_structure_df['E(N)'] = pricing_structure_df.apply(E_N, axis=1)
pricing_structure_df['E(Y)'] = pricing_structure_df.apply(E_Y, axis=1)
pricing_structure_df

Standard Insured ->  beta N: -1.67938083008443 ; beta Y: 7.27770480397245


,beta_N,beta_Y,E(N),E(Y)
Standard Insured,-1.679381,7.277705,0.186489,1447.661542
Zone D,0.233977,0.000000,0.235651,1447.661542
Zone E,0.406756,0.000000,0.280095,1447.661542
Zone F,0.415336,0.000000,0.282509,1447.661542
Zone A & B,-0.165548,0.000000,0.158037,1447.661542
Brand 2,0.055325,0.043495,0.197098,1512.016551
Brand 6,-0.117159,-0.071770,0.165872,1347.403572
Brand 12,-0.133241,0.127862,0.163226,1645.117198
Brand 14,-0.213623,-0.146288,0.150619,1250.648104
Brand 10 & 11 & 13,0.264279,0.024495,0.242901,1483.559992


Now computing the pure premium is just multiplying the two expected values:

In [5]:
pricing_structure_df['Pure Premium'] = pricing_structure_df['E(N)'] * pricing_structure_df['E(Y)']
pricing_structure_df

,beta_N,beta_Y,E(N),E(Y),Pure Premium
Standard Insured,-1.679381,7.277705,0.186489,1447.661542,269.973545
Zone D,0.233977,0.000000,0.235651,1447.661542,341.142808
Zone E,0.406756,0.000000,0.280095,1447.661542,405.483232
Zone F,0.415336,0.000000,0.282509,1447.661542,408.977522
Zone A & B,-0.165548,0.000000,0.158037,1447.661542,228.783562
Brand 2,0.055325,0.043495,0.197098,1512.016551,298.014855
Brand 6,-0.117159,-0.071770,0.165872,1347.403572,223.496230
Brand 12,-0.133241,0.127862,0.163226,1645.117198,268.525398
Brand 14,-0.213623,-0.146288,0.150619,1250.648104,188.371006
Brand 10 & 11 & 13,0.264279,0.024495,0.242901,1483.559992,360.357973


Computing the tariff is simply computing how much does the insurance goes up or down depending on feature:

In [6]:
std_tariff = pricing_structure_df.loc['Standard Insured'][4]

def tarrif(row):
    pp = row['Pure Premium']
    if row.name != "Standard Insured":
        return pp/std_tariff
    else:
        return pp
    
pricing_structure_df['Tariff'] = pricing_structure_df.apply(tarrif, axis=1)
pricing_structure_df

,beta_N,beta_Y,E(N),E(Y),Pure Premium,Tariff
Standard Insured,-1.679381,7.277705,0.186489,1447.661542,269.973545,269.973545
Zone D,0.233977,0.000000,0.235651,1447.661542,341.142808,1.263616
Zone E,0.406756,0.000000,0.280095,1447.661542,405.483232,1.501937
Zone F,0.415336,0.000000,0.282509,1447.661542,408.977522,1.514880
Zone A & B,-0.165548,0.000000,0.158037,1447.661542,228.783562,0.847430
Brand 2,0.055325,0.043495,0.197098,1512.016551,298.014855,1.103867
Brand 6,-0.117159,-0.071770,0.165872,1347.403572,223.496230,0.827845
Brand 12,-0.133241,0.127862,0.163226,1645.117198,268.525398,0.994636
Brand 14,-0.213623,-0.146288,0.150619,1250.648104,188.371006,0.697739
Brand 10 & 11 & 13,0.264279,0.024495,0.242901,1483.559992,360.357973,1.334790


Export to csv for analysis:

In [7]:
pricing_structure_df.to_csv("./out/part2-ex3-pricing_structure.csv")